## XGBoost
The hype. So I wanna join in and try the hype

## VotingClassifier (Soft and Hard)
It is the ultimate classifier for classical machine learning methods so why not try

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

In [16]:
# Input data before feature selection
input_data_before_fs = pd.read_csv('processed_train.csv', index_col=0)

# Input data after feature selection
input_data_after_fs = pd.read_csv('processed_train_after_feature.csv', index_col=0)

# Upsampling without feature selection

# Upsampling with feature selection

# Downsampling without feature selection

# Upsampling with feature selection


# List of all the input data
input_all = {
#     "normal_before_fs" : input_data_before_fs,
    "normal_after_fs" : input_data_after_fs
}

In [10]:
# Functions
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

def metric_consolidation(input_all, classifier, method = "cross_validation"):
    metrics = {'accuracy': 'accuracy',
               'roc_auc': make_scorer(multiclass_roc_auc_score, average='weighted'),
               'f1_weighted': 'f1_weighted'
              }
    
    for input_name, input_data in input_all.items():
        # split the data
        x_train, x_test, y_train, y_test = preprocessing(input_data)

        # fit the classifier to the training data
        classifier.fit(x_train, y_train)

        # apply all metrics to the classifier for cross_validation
        if method == "cross_validation":
            scores = tenfold(classifier, x_train, y_train, metric = metrics)
            print ("Metrics for %s: \n" %input_name)
            for metric in metrics:
                test_score_name = "test_" + metric
                test_score = scores[test_score_name]
                print ("%s Test Score: %0.2f +/- %0.2f" %(metric, test_score.mean()*100,
                                               test_score.std()*100))   
            print ("\n")
            
        if method == "test":
            y_pred = classifier.predict(x_test)
            accuracy = accuracy_score(y_test, y_pred)
            roc_score = multiclass_roc_auc_score(y_test, y_pred, average='weighted')
            f1_weighted = f1_score(y_test, y_pred, average='weighted')
            print(confusion_matrix(y_test, y_pred))
            
            metric_values = {'accuracy': accuracy,
                             'roc_auc': roc_score,
                             'f1_weighted': f1_weighted
                            }
            for metric in metrics:
                test_score = metric_values[metric]
                print ("%s Test Score: %0.2f +/- %0.2f" %(metric, test_score.mean()*100,
                                               test_score.std()*100)) 
            
def preprocessing(data):
    #Split data into variables types - boolean, categorical, continuous, ID
    bool_var = list(data.select_dtypes(['bool']))
    cont_var = list(data.select_dtypes(['float64']))
    cat_var = list(data.select_dtypes(['int64']))

    #Input Data can be from all except id details
    final_input_data = data[cat_var + cont_var + bool_var]
    
    x = final_input_data.loc[:, final_input_data.columns != 'Target'].values
    y = final_input_data['Target'].values
    y=y.astype('int')
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, 
                                                    random_state = 100 , stratify = y)
    
    return x_train, x_test, y_train, y_test

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import cross_validate

def tenfold(model, x, y, metric='accuracy'):
    kfold = StratifiedKFold(n_splits=10, random_state=100, shuffle=True)
    scores = cross_validate(model, x, y, cv=kfold, scoring=metric, 
                            return_train_score=True)
    return scores

# accuracy_mean = scores['test_score'].mean()
# accuracy_std = scores['train_score'].std()

In [12]:
# XGBoost
from xgboost import XGBClassifier

clf = XGBClassifier()
metric_consolidation(input_all, clf, method='test')

[[  9  12   0  34]
 [  6  31   6  68]
 [  3  15   8  63]
 [  4  27   3 455]]
accuracy Test Score: 67.61 +/- 0.00
f1_weighted Test Score: 62.19 +/- 0.00
roc_auc Test Score: 61.75 +/- 0.00


In [5]:
# Voting Classifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

clf1 = svm.SVC(C=0.1,decision_function_shape='ovo', kernel='linear', max_iter=-1, random_state=100)
clf2 = RandomForestClassifier(n_estimators=50, random_state=0)
clf3 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                                         max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
                                         min_samples_split=50, min_weight_fraction_leaf=0.0,
                                         presort=False, random_state=100, splitter='best')
clf4 = LogisticRegression(random_state=100, penalty = 'l1', C = 10**-1)
clf5 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                         random_state=0)


eclf = VotingClassifier(estimators=[('svm', clf1), ('rf', clf2), ('dt', clf3), ('lr', clf4), 
                                    ('bgt', clf5)], 
                        voting='hard')

In [7]:
# Voting Classifier Parameters
import warnings
warnings.filterwarnings('ignore')
voting_values = ['hard', 'soft']

for voting in voting_values:
#     svm = svm.SVC(C=0.1,decision_function_shape='ovo', kernel='linear', 
#                    max_iter=-1, random_state=100)
    dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                                             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
                                             min_samples_split=50, min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=100, splitter='best')
    lr = LogisticRegression(random_state=100, penalty = 'l1', C = 10**-1)

    rf = RandomForestClassifier(n_estimators=50, random_state=0)
    gbt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,random_state=0)
    bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                            random_state=100)
    
    
    eclf = VotingClassifier(estimators=[('rf', rf), ('dt', dt) , ('lr', lr), 
                                        ('gdt', gdt), ('bag', bag)], voting=voting)
#     eclf = VotingClassifier(estimators=[('rf', rf), ('bgt', clf5), ('bag', clf6)], 
#                             voting=voting)
    
    print ("For Voting Classifier with: \n voting: %s \n" %(voting))
    metric_consolidation(input_all, eclf)

For Voting Classifier with: 
 voting: hard 

Metrics for normal_before_fs: 

accuracy Test Score: 68.73 +/- 2.44
f1_weighted Test Score: 62.85 +/- 3.03
roc_auc Test Score: 63.94 +/- 2.43


For Voting Classifier with: 
 voting: soft 

Metrics for normal_before_fs: 

accuracy Test Score: 67.93 +/- 2.29
f1_weighted Test Score: 61.68 +/- 2.80
roc_auc Test Score: 62.69 +/- 2.13




In [11]:
# clf1 = svm.SVC(C=0.1,decision_function_shape='ovo', kernel='linear', 
#                max_iter=-1, random_state=100)
clf2 = RandomForestClassifier(n_estimators=50, random_state=0)
clf3 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                                         max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
                                         min_samples_split=50, min_weight_fraction_leaf=0.0,
                                         presort=False, random_state=100, splitter='best')
clf4 = LogisticRegression(random_state=100, penalty = 'l1', C = 10**-1)
clf5 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, 
                                         random_state=0)
clf6 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), 
                         n_estimators=100, random_state=100)


eclf = VotingClassifier(estimators=[('rf', clf2), ('dt', clf3) , ('lr', clf4), 
                                    ('bgt', clf5), ('bag', clf6)], voting='hard')

metric_consolidation(input_all, eclf, method='test')

[[ 10  11   1  33]
 [  6  29   5  71]
 [  4  17   3  65]
 [  5  21   8 455]]
accuracy Test Score: 66.80 +/- 0.00
f1_weighted Test Score: 60.84 +/- 0.00
roc_auc Test Score: 60.84 +/- 0.00


In [13]:
#XGBoost Parameters
learning_rate_values = [0.1, 0.5, 1]
n_estimators_values = [50, 100, 150]

for learning_rate in learning_rate_values:
    for n_estimators in n_estimators_values:
        clf = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators)

        print ("For XGBoost with: \n learning rate: %s \n n_estimators: %s \n"
              %(learning_rate, n_estimators))
        
        metric_consolidation(input_all, clf)

For XGBoost with: 
 learning rate: 0.1 
 n_estimators: 50 

Metrics for normal_before_fs: 

accuracy Test Score: 68.42 +/- 2.18
f1_weighted Test Score: 61.36 +/- 2.50
roc_auc Test Score: 61.51 +/- 2.05


For XGBoost with: 
 learning rate: 0.1 
 n_estimators: 100 

Metrics for normal_before_fs: 

accuracy Test Score: 67.98 +/- 2.31
f1_weighted Test Score: 61.96 +/- 2.47
roc_auc Test Score: 63.33 +/- 1.78


For XGBoost with: 
 learning rate: 0.1 
 n_estimators: 150 

Metrics for normal_before_fs: 

accuracy Test Score: 67.89 +/- 1.90
f1_weighted Test Score: 62.60 +/- 2.14
roc_auc Test Score: 64.09 +/- 1.63


For XGBoost with: 
 learning rate: 0.5 
 n_estimators: 50 

Metrics for normal_before_fs: 

accuracy Test Score: 67.66 +/- 2.23
f1_weighted Test Score: 63.67 +/- 2.32
roc_auc Test Score: 65.54 +/- 1.91


For XGBoost with: 
 learning rate: 0.5 
 n_estimators: 100 

Metrics for normal_before_fs: 

accuracy Test Score: 66.18 +/- 2.11
f1_weighted Test Score: 62.73 +/- 2.07
roc_auc Test S

In [17]:
clf = XGBClassifier(learning_rate=0.5, n_estimators=50)

metric_consolidation(input_all, clf, method='test')

[[  9  13   3  30]
 [ 11  29  12  59]
 [  5  21  12  51]
 [  6  26  17 440]]
accuracy Test Score: 65.86 +/- 0.00
f1_weighted Test Score: 62.14 +/- 0.00
roc_auc Test Score: 63.77 +/- 0.00
